<a href="https://colab.research.google.com/github/parsa-abbasi/Sentiment-Analysis/blob/master/SentiPers/BinaryClassifier/BaseLine/GoogleColab/FullTranslate-NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hazm
!pip install stopwords_guilannlp

    100% |████████████████████████████████| 317kB 23.6MB/s 
    100% |████████████████████████████████| 1.4MB 6.0MB/s 
    100% |████████████████████████████████| 235kB 16.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
Successfully built nltk libwapiti
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [0]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import Pipeline
from hazm import *
from stopwords_guilannlp import stopwords_output
import pandas as pd
import numpy as np

In [69]:
from google.colab import files
uploaded = files.upload()

Saving x_test.csv to x_test (2).csv
Saving x_train.csv to x_train (2).csv
Saving y_test.csv to y_test (2).csv
Saving y_train.csv to y_train (2).csv


In [78]:
x_train = pd.Series.from_csv('x_train (2).csv', sep='\t')
x_test = pd.Series.from_csv('x_test (2).csv', sep='\t')
y_train = pd.Series.from_csv('y_train (2).csv', sep='\t', header=0)
y_test = pd.Series.from_csv('y_test (2).csv', sep='\t', header=0)
x_train = x_train.iloc[1:, ]
x_test = x_test.iloc[1:, ]
print('x_train shape: ', x_train.shape)
print('x_test shape: ', x_test.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

x_train shape:  (11122,)
x_test shape:  (1854,)
y_train shape:  (11122,)
y_test shape:  (1854,)


/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:3727: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


In [79]:
# See the data number of each category 
from collections import Counter
cnt = Counter(y_train)
cnt = dict(cnt)
print(cnt)

{1: 3246, 2: 1976, 0: 4818, -1: 1026, -2: 56}


In [80]:
binary_y_train = []
binary_y_test = []
binary_x_train = []
binary_x_test = []
for i, y in enumerate(y_train):
  if y != 0:
    if y > 0:
      binary_y_train.append(1)
      binary_x_train.append(x_train[i])
    else:
      binary_y_train.append(0)
      binary_x_train.append(x_train[i])
  else:
    binary_y_train.append(0)
    binary_x_train.append(x_train[i])
      
for i, y in enumerate(y_test):
  if y != 0:
    if y > 0:
      binary_y_test.append(1)
      binary_x_test.append(x_test[i])
    else:
      binary_y_test.append(0)
      binary_x_test.append(x_test[i])
  else:
    binary_y_test.append(0)
    binary_x_test.append(x_test[i])
      

x_train = np.asarray(binary_x_train)
x_test = np.asarray(binary_x_test)
y_train = np.asarray(binary_y_train)
y_test = np.asarray(binary_y_test)
print('x_train shape: ', x_train.shape)
print('x_test shape: ', x_test.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)

x_train shape:  (11122,)
x_test shape:  (1854,)
y_train shape:  (11122,)
y_test shape:  (1854,)


In [81]:
# See the data number of each category 
from collections import Counter
cnt = Counter(y_train)
cnt = dict(cnt)
print(cnt)

{1: 5222, 0: 5900}


In [0]:
# Make stop word set
stop_set = stopwords_output("Persian", "set")


# When building the vocabulary ignore terms that have a document frequency strictly lower than
# the given threshold. This value is also called cut-off in the literature.
min_df = 1


# Tokenize function used in Vectorizer
def tokenize(text):
    return word_tokenize(text)


In [83]:
# (Multinomial) Naive Bayes Model
text_clf = Pipeline([('vect', CountVectorizer(tokenizer=tokenize,
                                              analyzer='word', ngram_range=(1, 2), min_df=min_df, lowercase=False)),
                     ('tfidf', TfidfTransformer(sublinear_tf=True)),
                     ('clf', MultinomialNB())])
text_clf = text_clf.fit(x_train, y_train)
naive_score = text_clf.score(x_test, y_test)
print('Naive Bayes Model: ', naive_score)
predict = text_clf.predict(x_test)


Naive Bayes Model:  0.7923408845738943


In [84]:
# SGD (Stochastic Gradient Descent) Model
text_clf_sgd = Pipeline([('vect', CountVectorizer(tokenizer=tokenize,
                                                  analyzer='word', ngram_range=(1, 2), min_df=min_df, lowercase=False)),
                         ('tfidf', TfidfTransformer(sublinear_tf=True)),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                                   alpha=1e-3, max_iter=5))])
text_clf_sgd = text_clf_sgd.fit(x_train, y_train)
sgd_score = text_clf_sgd.score(x_test, y_test)
print('SGD Model: ', sgd_score)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


SGD Model:  0.7076591154261057


In [85]:
# Linear Support Vector Machine Model
text_clf_linear_svc = Pipeline([('vect', CountVectorizer(tokenizer=tokenize,
                                                         analyzer='word', ngram_range=(1, 2),
                                                         min_df=min_df, lowercase=False)),
                                ('tfidf', TfidfTransformer(sublinear_tf=True)),
                                ('clf-svm', LinearSVC(loss='hinge', penalty='l2',
                                                      max_iter=5))])

text_clf_linear_svc = text_clf_linear_svc.fit(x_train, y_train)
linear_svc_score = text_clf_linear_svc.score(x_test, y_test)
print('Linear SVC Model: ', linear_svc_score)


Linear SVC Model:  0.8058252427184466


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
